In [18]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import PromptTemplate

In [3]:
# init_library_db.py
import sqlite3

connection = sqlite3.connect("library.db")
cursor = connection.cursor()

# Create the table
cursor.execute("""
CREATE TABLE IF NOT EXISTS library (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    year_published INTEGER,
    rating REAL,
    available INTEGER
)
""")

# Insert sample data
sample_books = [
    ("The Great Gatsby", "F. Scott Fitzgerald", "Fiction", 1925, 4.4, 1),
    ("1984", "George Orwell", "Dystopian", 1949, 4.7, 1),
    ("To Kill a Mockingbird", "Harper Lee", "Fiction", 1960, 4.8, 1)
]

cursor.executemany("""
INSERT INTO library (title, author, genre, year_published, rating, available)
VALUES (?, ?, ?, ?, ?, ?)
""", sample_books)

connection.commit()
connection.close()

print("library.db initialized with 'library' table and sample data.")


library.db initialized with 'library' table and sample data.


In [4]:
import sqlite3
from langchain.prompts import PromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser

i = input('nl: ')

prompt = PromptTemplate(
    input_variables=["nl_query"],
    template="""
You are an assistant that converts English to SQL.
Assume the SQLite table "library" with columns:
(id,title, author, genre, year_published, rating, available)

Convert the following question to a SQL query ONLY (no explanation):
Question: {nl_query}
"""
)

llm = ChatNVIDIA()
chain = prompt | llm | StrOutputParser()

response = chain.invoke({"nl_query": i})
sql_query = response.strip().split('\n')[0]

sql_query


"INSERT INTO library (title, author, genre, year_published, rating, available) VALUES ('Harry Potter and the Philosophers Stone', 'J.K. Rowling', 'Fantasy', 1997, 4.2, 1);"

In [5]:
connection = sqlite3.connect("library.db")
cursor = connection.cursor()

try:
    cursor.execute(sql_query)
    cursor.execute("SELECT * FROM library")
    rows = cursor.fetchall()
except sqlite3.OperationalError as e:
    print(f"SQL Error: {e}")
    print(f"Problematic query: {sql_query}")
    rows = []

connection.close()

result = "\n".join([str(row) for row in rows]) or "No results found."
print(result)

(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction', 1925, 4.4, 1)
(2, '1984', 'George Orwell', 'Dystopian', 1949, 4.7, 1)
(3, 'To Kill a Mockingbird', 'Harper Lee', 'Fiction', 1960, 4.8, 1)
(4, 'Harry Potter and the Philosophers Stone', 'J.K. Rowling', 'Fantasy', 1997, 4.2, 1)


In [6]:
for row in rows:
    print(row)

(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction', 1925, 4.4, 1)
(2, '1984', 'George Orwell', 'Dystopian', 1949, 4.7, 1)
(3, 'To Kill a Mockingbird', 'Harper Lee', 'Fiction', 1960, 4.8, 1)
(4, 'Harry Potter and the Philosophers Stone', 'J.K. Rowling', 'Fantasy', 1997, 4.2, 1)


In [7]:
import sqlite3

connection = sqlite3.connect("library.db")
cursor = connection.cursor()

cursor.execute("SELECT * FROM library")
rows = cursor.fetchall()

for row in rows:
    print(row)

connection.close()


(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction', 1925, 4.4, 1)
(2, '1984', 'George Orwell', 'Dystopian', 1949, 4.7, 1)
(3, 'To Kill a Mockingbird', 'Harper Lee', 'Fiction', 1960, 4.8, 1)
